### Set Up Packages and Global Settings

In [1]:
import pytz
import random
import time

from datetime import datetime

import fastf1 as ff1
import pandas as pd

pd.set_option("display.max_columns", None)

### Explore Schedule

In [ ]:
schedule = ff1.get_event_schedule(1994)
schedule.RoundNumber.to_list()

### Explore Session Object

In [ ]:
session_quali = ff1.get_session(2020, 1, "Q")

session_quali.load()
session_quali.event

In [ ]:
session_quali = ff1.get_session(2020, 1, 'R')

session_quali.load()
session_quali.results

### Explore Laps

In [ ]:
lap_data = session_number.laps
print(lap_data)

In [ ]:
lap_data.pick_team('McLaren').pick_fastest()

In [ ]:
lap_data.pick_drivers(['PIA', 'NOR'])

### Compare DriverId field to names

In [ ]:
df_all = pd.DataFrame()

list_years = list(range(1994, 1995 + 1))

max_retries = 5

for year in list_years:
    print(f"Pulling data for {year}")

    schedule = ff1.get_event_schedule(year)
    rounds_all = schedule.RoundNumber.to_list()
    rounds_races = [round for round in rounds_all if round > 0]

    for round in rounds_races:
        for attempt in range(max_retries):
            try:
                now_utc = datetime.now(pytz.timezone("UTC"))
                now_cst = now_utc.astimezone(pytz.timezone("America/Chicago"))

                print(f"Pulling data for round {round} at {now_cst}...")

                session_quali = ff1.get_session(year, round, "Q")
                session_quali.load()
                df_quali = session_quali.results[["DriverId", "LastName", "FirstName"]]

                df_all = df_all.append(df_quali, ignore_index=True)
                time.sleep(5)

                session_race = ff1.get_session(year, round, "R")
                session_race.load()
                df_race = session_race.results[["DriverId", "LastName", "FirstName"]]

                df_all = df_all.append(df_race, ignore_index=True)
                time.sleep(5)

                df_all = df_all.drop_duplicates().reset_index(drop=True)

                # If no exception was raised, break the loop
                break
            except Exception as e:
                wait_time = (2**attempt) + random.random()
                print(
                    f"Could not load data for round {round} due to {e}. Retrying in {wait_time} seconds."
                )
                time.sleep(wait_time)
                continue

df_unique = df_all.sort_values(by="DriverId")

### Develop Approach for Results Object

In [21]:
session_quali = ff1.get_session(2023, 1, "Q")

session_quali.load()
df_raw = session_quali.results[
    ["Q1", "Q2", "Q3", "DriverId", "LastName", "FirstName", "TeamName", "Position"]
]

df_reshaped = df_raw.melt(
    id_vars=["DriverId", "LastName", "FirstName", "TeamName", "Position"],
    value_vars=["Q1", "Q2", "Q3"],
    var_name="session",
    value_name="time",
)

df_reshaped.columns = df_reshaped.columns.str.lower()

df_reshaped["originalposition"] = df_reshaped["position"]

df_reshaped["position"] = df_reshaped.groupby("session")["time"].rank(
    method="min", ascending=True
)

df_reshaped["position"] = df_reshaped["position"].fillna(
    df_reshaped["originalposition"]
)

df_final = df_reshaped.drop(columns=["originalposition"])

df_final

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '55', '14', '63', '44', '18', '31', '27', '4', '77', '24', '22', '23', '2', '20', '81', '21', '10']


,driverid,lastname,firstname,teamname,position,session,time
0,max_verstappen,Verstappen,Max,Red Bull Racing,7.0,Q1,0 days 00:01:31.295000
1,perez,Perez,Sergio,Red Bull Racing,10.0,Q1,0 days 00:01:31.479000
2,leclerc,Leclerc,Charles,Ferrari,3.0,Q1,0 days 00:01:31.094000
3,sainz,Sainz,Carlos,Ferrari,1.0,Q1,0 days 00:01:30.993000
4,alonso,Alonso,Fernando,Aston Martin,4.0,Q1,0 days 00:01:31.158000
5,russell,Russell,George,Mercedes,2.0,Q1,0 days 00:01:31.057000
6,hamilton,Hamilton,Lewis,Mercedes,13.0,Q1,0 days 00:01:31.543000
7,stroll,Stroll,Lance,Aston Martin,5.0,Q1,0 days 00:01:31.184000
8,ocon,Ocon,Esteban,Alpine,12.0,Q1,0 days 00:01:31.508000
9,hulkenberg,Hulkenberg,Nico,Haas F1 Team,6.0,Q1,0 days 00:01:31.204000


In [18]:
df_final

Index(['driverid', 'lastname', 'firstname', 'teamname', 'position', 'session',
       'time'],
      dtype='object')